# 🔍 Análisis Avanzado de Issues de SonarCloud en Proyectos Estudiantiles

## 🎯 Objetivo
Análisis exhaustivo y multidimensional de **7,844 issues detallados** extraídos de SonarCloud de **60 estudiantes** en dos asignaturas: **"Programación Aplicada I" (AP1)** y **"Programación Aplicada II" (AP2)"**.

## 📊 Dataset
- **7,844 issues detallados** de SonarCloud
- **60 estudiantes** de programación aplicada
- **Dos asignaturas consecutivas**: AP1 → AP2
- **Múltiples métricas**: tipo, severidad, deuda técnica, ubicación, reglas

## 🔬 Enfoque de Análisis
Este notebook combina:
- **Análisis estadístico riguroso** para identificar patrones significativos
- **Minería de datos educativos** para insights pedagógicos
- **Visualizaciones interactivas** para exploración dinámica
- **Modelado predictivo** para sistemas de alerta temprana
- **Recomendaciones accionables** basadas en evidencia

---

## 📋 Índice de Contenido

1. **Data Loading & Preprocessing** - Carga y limpieza de datos
2. **Exploratory Data Analysis** - Análisis exploratorio integral
3. **Rule Analysis & Categorization** - Análisis de reglas violadas
4. **Technical Debt Analysis** - Análisis de deuda técnica
5. **Issue Type Analysis** - Análisis por tipos de issues
6. **Student Profiling & Clustering** - Perfilado y clustering de estudiantes
7. **Temporal & Evolution Analysis** - Análisis temporal y evolutivo
8. **Location & Context Analysis** - Análisis de ubicación y contexto
9. **Message Text Mining Analysis** - Minería de texto en mensajes
10. **Advanced Statistical Analysis** - Análisis estadístico avanzado
11. **Specialized Visualizations** - Visualizaciones especializadas
12. **Predictive Modeling** - Modelado predictivo
13. **Educational Insights & Recommendations** - Insights educativos y recomendaciones

---

# 1. 📥 Data Loading and Preprocessing

En esta sección cargamos y preprocesamos el dataset de issues de SonarCloud, realizamos limpieza de datos y creamos características derivadas para el análisis.

In [ ]:
# ===================================================================
# IMPORTS Y CONFIGURACIÓN INICIAL
# ===================================================================

# Librerías básicas para manejo de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Librerías para análisis estadístico
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu, wilcoxon, shapiro
from statsmodels.stats.contingency_tables import mcnemar
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, adjusted_rand_score

# Librerías para machine learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Librerías para procesamiento de texto
import re
from collections import Counter
from wordcloud import WordCloud
import nltk
from textblob import TextBlob

# Librerías para análisis de redes
import networkx as nx

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configuración de warnings
import warnings
warnings.filterwarnings('ignore')

# Configuración de plotly para notebooks
import plotly.io as pio
pio.renderers.default = "notebook"

print("✅ Librerías importadas exitosamente")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 Plotly disponible para visualizaciones interactivas")

In [ ]:
# ===================================================================
# CARGA DE DATOS
# ===================================================================

# Cargar el dataset de issues detallados
print("🔄 Cargando dataset de issues...")
issues_df = pd.read_csv('../data/issues_detallados_latest.csv')

print(f"✅ Dataset cargado exitosamente")
print(f"📊 Dimensiones: {issues_df.shape[0]:,} issues × {issues_df.shape[1]} columnas")
print(f"👥 Estudiantes únicos: {issues_df['student_id'].nunique()}")
print(f"📚 Asignaturas: {', '.join(issues_df['assignment'].unique())}")

# Mostrar información básica del dataset
print("\n" + "="*60)
print("📋 INFORMACIÓN GENERAL DEL DATASET")
print("="*60)
print(f"Total de issues: {len(issues_df):,}")
print(f"Estudiantes únicos: {issues_df['student_id'].nunique()}")
print(f"Proyectos únicos: {issues_df['project_key'].nunique()}")
print(f"Reglas únicas violadas: {issues_df['rule'].nunique()}")
print(f"Tipos de issues: {', '.join(issues_df['type'].unique())}")
print(f"Niveles de severidad: {', '.join(issues_df['severity'].unique())}")

# Mostrar las primeras filas
print("\n" + "="*60)
print("🔍 MUESTRA DE DATOS")
print("="*60)
issues_df.head()

In [ ]:
# ===================================================================
# ANÁLISIS DE CALIDAD DE DATOS Y LIMPIEZA
# ===================================================================

print("🔍 ANÁLISIS DE CALIDAD DE DATOS")
print("="*60)

# Verificar valores faltantes
missing_values = issues_df.isnull().sum()
missing_percentage = (missing_values / len(issues_df)) * 100

print("📊 Valores faltantes por columna:")
for col, missing in missing_values.items():
    if missing > 0:
        print(f"   {col}: {missing:,} ({missing_percentage[col]:.2f}%)")

# Información detallada de las columnas
print(f"\n📋 INFORMACIÓN DETALLADA DE COLUMNAS")
print("="*60)
print(issues_df.info())

# Análisis de duplicados
print(f"\n🔄 ANÁLISIS DE DUPLICADOS")
print("="*60)
duplicates = issues_df.duplicated().sum()
print(f"Issues duplicados: {duplicates}")

if duplicates > 0:
    print("Eliminando duplicados...")
    issues_df = issues_df.drop_duplicates()
    print(f"Dataset después de eliminar duplicados: {len(issues_df):,} issues")

# Convertir tipos de datos apropiados
print(f"\n🔧 CONVERSIÓN DE TIPOS DE DATOS")
print("="*60)

# Convertir fechas
date_columns = ['creation_date', 'update_date']
for col in date_columns:
    if col in issues_df.columns:
        issues_df[col] = pd.to_datetime(issues_df[col], errors='coerce')
        print(f"✅ {col} convertido a datetime")

# Convertir effort y debt a minutos numéricos
for col in ['effort', 'debt']:
    if col in issues_df.columns:
        # Extraer números de strings como "5min", "10min", etc.
        issues_df[f'{col}_minutes'] = issues_df[col].str.extract('(\d+)').astype(float)
        issues_df[f'{col}_minutes'] = issues_df[f'{col}_minutes'].fillna(0)
        print(f"✅ {col} convertido a minutos numéricos")

# Verificar rangos de datos
print(f"\n📊 RANGOS DE DATOS")
print("="*60)
print(f"Rango de fechas de creación: {issues_df['creation_date'].min()} a {issues_df['creation_date'].max()}")
print(f"Rango de líneas de código: {issues_df['line'].min()} a {issues_df['line'].max()}")
print(f"Effort range: {issues_df['effort_minutes'].min()} a {issues_df['effort_minutes'].max()} minutos")
print(f"Debt range: {issues_df['debt_minutes'].min()} a {issues_df['debt_minutes'].max()} minutos")

print("\n✅ Limpieza de datos completada")

In [ ]:
# ===================================================================
# CREACIÓN DE CARACTERÍSTICAS DERIVADAS
# ===================================================================

print("🔧 CREACIÓN DE CARACTERÍSTICAS DERIVADAS")
print("="*60)

# 1. Extraer tipo de archivo de component
issues_df['file_extension'] = issues_df['component'].str.extract(r'\.([^.]+)$')
issues_df['file_extension'] = issues_df['file_extension'].fillna('unknown')

# 2. Extraer namespace/carpeta principal
issues_df['namespace'] = issues_df['component'].str.extract(r':([^/]+)')
issues_df['folder_level_1'] = issues_df['component'].str.extract(r':([^/]+/[^/]+)')

# 3. Categorizar familias de reglas
def categorize_rule_family(rule):
    if pd.isna(rule):
        return 'unknown'
    elif rule.startswith('external_roslyn:CS'):
        return 'roslyn_csharp'
    elif rule.startswith('csharpsquid:S'):
        return 'sonarqube_csharp'
    elif rule.startswith('css:S'):
        return 'css'
    elif rule.startswith('typescript:S'):
        return 'typescript'
    elif rule.startswith('javascript:S'):
        return 'javascript'
    else:
        return 'other'

issues_df['rule_family'] = issues_df['rule'].apply(categorize_rule_family)

# 4. Mapear severidad a valores numéricos para análisis
severity_mapping = {
    'CRITICAL': 4,
    'MAJOR': 3,
    'MINOR': 2,
    'INFO': 1
}
issues_df['severity_numeric'] = issues_df['severity'].map(severity_mapping)

# 5. Crear indicadores binarios para tipos de issues
for issue_type in issues_df['type'].unique():
    issues_df[f'is_{issue_type.lower()}'] = (issues_df['type'] == issue_type).astype(int)

# 6. Extraer año y mes de creación
issues_df['creation_year'] = issues_df['creation_date'].dt.year
issues_df['creation_month'] = issues_df['creation_date'].dt.month

# 7. Calcular métricas agregadas por estudiante
print("📊 Calculando métricas por estudiante...")

student_metrics = issues_df.groupby(['student_id', 'assignment']).agg({
    'issue_key': 'count',
    'severity_numeric': ['mean', 'sum'],
    'effort_minutes': 'sum',
    'debt_minutes': 'sum',
    'type': lambda x: x.value_counts().to_dict(),
    'rule_family': lambda x: x.value_counts().to_dict(),
    'file_extension': 'nunique'
}).round(2)

student_metrics.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in student_metrics.columns]
student_metrics = student_metrics.rename(columns={
    'issue_key_count': 'total_issues',
    'severity_numeric_mean': 'avg_severity',
    'severity_numeric_sum': 'total_severity_score',
    'effort_minutes_sum': 'total_effort_minutes',
    'debt_minutes_sum': 'total_debt_minutes',
    'file_extension_nunique': 'files_with_issues'
})

# 8. Crear dataset de métricas por estudiante en formato amplio
student_summary = issues_df.pivot_table(
    index=['student_id', 'nombre', 'assignment'],
    values='issue_key',
    columns='type',
    aggfunc='count',
    fill_value=0
).reset_index()

# Aplanar nombres de columnas
student_summary.columns.name = None

print("✅ Características derivadas creadas:")
print(f"   📁 Extensiones de archivo: {issues_df['file_extension'].nunique()} tipos")
print(f"   🏗️ Familias de reglas: {issues_df['rule_family'].nunique()} familias")
print(f"   📊 Métricas por estudiante calculadas para {student_summary.shape[0]} registros")

# Mostrar ejemplo de métricas por estudiante
print(f"\n📋 EJEMPLO DE MÉTRICAS POR ESTUDIANTE")
print("="*60)
student_summary.head()

# 2. 📊 Exploratory Data Analysis (EDA)

En esta sección realizamos un análisis exploratorio comprehensivo de los datos de issues, examinando distribuciones, patrones temporales y características principales del dataset.

In [ ]:
# ===================================================================
# DISTRIBUCIÓN GENERAL DE ISSUES
# ===================================================================

print("📊 ANÁLISIS DE DISTRIBUCIÓN GENERAL")
print("="*60)

# Estadísticas por asignatura
assignment_stats = issues_df.groupby('assignment').agg({
    'issue_key': 'count',
    'student_id': 'nunique',
    'severity_numeric': 'mean',
    'effort_minutes': 'sum',
    'debt_minutes': 'sum'
}).round(2)

print("Estadísticas por asignatura:")
print(assignment_stats)

# Distribución por tipo de issue
print(f"\n📋 DISTRIBUCIÓN POR TIPO DE ISSUE")
type_dist = issues_df['type'].value_counts()
type_pct = issues_df['type'].value_counts(normalize=True) * 100

print("Conteo y porcentaje por tipo:")
for issue_type in type_dist.index:
    print(f"   {issue_type}: {type_dist[issue_type]:,} ({type_pct[issue_type]:.1f}%)")

# Distribución por severidad
print(f"\n⚠️ DISTRIBUCIÓN POR SEVERIDAD")
severity_dist = issues_df['severity'].value_counts()
severity_pct = issues_df['severity'].value_counts(normalize=True) * 100

print("Conteo y porcentaje por severidad:")
for severity in ['CRITICAL', 'MAJOR', 'MINOR', 'INFO']:
    if severity in severity_dist.index:
        print(f"   {severity}: {severity_dist[severity]:,} ({severity_pct[severity]:.1f}%)")

# Crear visualización de distribuciones
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📊 Distribución General de Issues', fontsize=16, fontweight='bold')

# 1. Issues por asignatura
assignment_counts = issues_df['assignment'].value_counts()
axes[0,0].bar(assignment_counts.index, assignment_counts.values, color=['#1f77b4', '#ff7f0e'])
axes[0,0].set_title('Issues por Asignatura')
axes[0,0].set_ylabel('Número de Issues')
for i, v in enumerate(assignment_counts.values):
    axes[0,0].text(i, v + 50, f'{v:,}', ha='center', fontweight='bold')

# 2. Issues por tipo
type_counts = issues_df['type'].value_counts()
colors = plt.cm.Set3(np.linspace(0, 1, len(type_counts)))
axes[0,1].bar(range(len(type_counts)), type_counts.values, color=colors)
axes[0,1].set_title('Issues por Tipo')
axes[0,1].set_ylabel('Número de Issues')
axes[0,1].set_xticks(range(len(type_counts)))
axes[0,1].set_xticklabels(type_counts.index, rotation=45, ha='right')
for i, v in enumerate(type_counts.values):
    axes[0,1].text(i, v + 50, f'{v:,}', ha='center', fontweight='bold')

# 3. Issues por severidad
severity_order = ['CRITICAL', 'MAJOR', 'MINOR', 'INFO']
severity_counts = issues_df['severity'].value_counts().reindex(severity_order)
severity_colors = ['#d62728', '#ff7f0e', '#ffbb78', '#2ca02c']
axes[1,0].bar(severity_counts.index, severity_counts.values, color=severity_colors)
axes[1,0].set_title('Issues por Severidad')
axes[1,0].set_ylabel('Número de Issues')
axes[1,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(severity_counts.values):
    axes[1,0].text(i, v + 50, f'{v:,}', ha='center', fontweight='bold')

# 4. Issues por estudiante (distribución)
issues_per_student = issues_df.groupby('student_id').size()
axes[1,1].hist(issues_per_student, bins=20, color='skyblue', alpha=0.7, edgecolor='black')
axes[1,1].set_title('Distribución de Issues por Estudiante')
axes[1,1].set_xlabel('Número de Issues por Estudiante')
axes[1,1].set_ylabel('Número de Estudiantes')
axes[1,1].axvline(issues_per_student.mean(), color='red', linestyle='--', 
                  label=f'Media: {issues_per_student.mean():.1f}')
axes[1,1].legend()

plt.tight_layout()
plt.show()

# Estadísticas descriptivas
print(f"\n📈 ESTADÍSTICAS DESCRIPTIVAS")
print("="*60)
print(f"Issues por estudiante:")
print(f"   Media: {issues_per_student.mean():.1f}")
print(f"   Mediana: {issues_per_student.median():.1f}")
print(f"   Desviación estándar: {issues_per_student.std():.1f}")
print(f"   Mínimo: {issues_per_student.min()}")
print(f"   Máximo: {issues_per_student.max()}")

In [ ]:
# ===================================================================
# ANÁLISIS TEMPORAL DE ISSUES
# ===================================================================

print("📅 ANÁLISIS TEMPORAL DE ISSUES")
print("="*60)

# Análisis por fecha de creación
if 'creation_date' in issues_df.columns:
    issues_df['creation_date_only'] = issues_df['creation_date'].dt.date
    
    # Issues por día
    daily_issues = issues_df.groupby(['creation_date_only', 'assignment']).size().reset_index(name='count')
    
    # Crear visualización temporal
    fig = px.line(daily_issues, x='creation_date_only', y='count', color='assignment',
                  title='📅 Evolución Temporal de Issues por Asignatura',
                  labels={'creation_date_only': 'Fecha de Creación', 'count': 'Número de Issues'})
    fig.update_layout(height=500)
    fig.show()
    
    # Estadísticas temporales por asignatura
    temporal_stats = issues_df.groupby('assignment')['creation_date'].agg(['min', 'max', 'count'])
    print("Estadísticas temporales por asignatura:")
    print(temporal_stats)
    
    # Análisis por mes
    issues_df['year_month'] = issues_df['creation_date'].dt.to_period('M')
    monthly_issues = issues_df.groupby(['year_month', 'assignment']).size().reset_index(name='count')
    monthly_issues['year_month_str'] = monthly_issues['year_month'].astype(str)
    
    print(f"\nIssues por mes y asignatura:")
    monthly_pivot = monthly_issues.pivot(index='year_month_str', columns='assignment', values='count').fillna(0)
    print(monthly_pivot)

# Análisis de status de issues
print(f"\n🔄 ANÁLISIS DE STATUS DE ISSUES")
print("="*60)
status_dist = issues_df['status'].value_counts()
status_pct = issues_df['status'].value_counts(normalize=True) * 100

print("Distribución de status:")
for status in status_dist.index:
    print(f"   {status}: {status_dist[status]:,} ({status_pct[status]:.1f}%)")

# Cross-tabulation de assignment vs tipo de issue
print(f"\n📊 CROSSTAB: ASIGNATURA vs TIPO DE ISSUE")
print("="*60)
crosstab_assignment_type = pd.crosstab(issues_df['assignment'], issues_df['type'], margins=True)
print(crosstab_assignment_type)

# Porcentajes por fila
print(f"\nPorcentajes por asignatura:")
crosstab_pct = pd.crosstab(issues_df['assignment'], issues_df['type'], normalize='index') * 100
print(crosstab_pct.round(1))

# Visualización de heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(crosstab_pct, annot=True, fmt='.1f', cmap='YlOrRd', 
            cbar_kws={'label': 'Porcentaje'})
plt.title('🔥 Heatmap: Distribución de Tipos de Issues por Asignatura (%)')
plt.xlabel('Tipo de Issue')
plt.ylabel('Asignatura')
plt.tight_layout()
plt.show()